In [1]:
import h5py
import os
import numpy as np
import json

DATA_DIR = "../data/"
RAW_DIR = os.path.join(DATA_DIR,"raw")

## Caution
아래의 스크립트를 실행하기 위해서는 다음과 같은 폴더 구성을 지켜주셔야 합니다. 
```
|- data/
     |- raw/
         |- train.chunk.01~09
         |- test.chunk.01~02
         |- dev.chunk.01
         |- cate1.json
```

이 코드들은 간단히 데이터를 까보기 위함일 뿐, 그렇게 의미있진 않을 겁니다. 1~3에서는
이 데이터를 어떻게 작업할지를 고민해 보았습니다.

저희가 가진 고민 포인트는 
> 큰 카카오 데이터를 램이 작은 컴퓨터에서 어떻게 효율적으로 처리할 것인가?

였습니다. 저희가 내린 결론은 h5파일을 like Pandas 스타일로 처리할 수 있는 
wrapper 클래스를 만들자입니다.

어떤 식으로 구성하였는지 궁금하시면 아래의 
`3) 데이터 로더 만들기.ipynb`부터 참고해주세요

## Objective

    우리의 카카오가 어떤 데이터를 우리에게 넘겨주셨는지 한번 보아요.
    주어진 데이터가 어떤 꼴인지를 확인 좀 해봅시다

In [2]:
f = h5py.File(os.path.join(RAW_DIR,"train.chunk.01"),'r')
print(list(f['train'].keys()))

['bcateid', 'brand', 'dcateid', 'img_feat', 'maker', 'mcateid', 'model', 'pid', 'price', 'product', 'scateid', 'updttm']


## 주어진 데이터셋

| 컬럼  | 타입  | 설명  | 예시|
|---|---|---| ---|
| pid  | string | 상품 ID | O4486751463 |
| product  | string  | 상품명  | 직소퍼즐 - 1000조각 바다거북의 여행 (PL1275) |
| brand  | string  | 브랜드명  | 퍼즐라이프 |
| model  | string  | 정제된 상품명  | 퍼즐라이프 직소퍼즐 바다거북의 여행 |
| maker  | string  | 제조사   | ㈜크리스패션 |
| price  | int32  | 가격 | 16520 |
| updttm |  string | 상품정보업데이트 시간 | 20180227091029 |
| bcateid | int32  | 대카테고리 ID  | 1 |
| mcateid | int32  | 중카테고리 ID  | 1 |
| scateid | int32  | 소카테고리 ID  | 2 |
| dcateid | int32  | 세카테고리 ID  | -1 |
| img_feat | float32 | ResNet50(imagenet)의 모델의 출력  |array([...,..]) |

In [3]:
def list_up_string_in_file(column_name, nums=20):
    # h5py의 column_name을 열어보는 코드
    return list(map(lambda x: x.decode('utf-8'), f['train'][column_name][:nums]))

### pid

상품 ID에는 정해진 길이의 string으로 된 구조가 있는 ID이다.

가설 1) 
> 브랜드명, 상품명, 제조사, 카테고리 등의 정보로 조합되었을 것이다.

In [4]:
list_up_string_in_file(column_name="pid", nums=30)

['O4486751463',
 'P3307178849',
 'R4424255515',
 'F3334315393',
 'N731678492',
 'J4094617432',
 'V4742097320',
 'Z4154445264',
 'L2121928457',
 'J2944368118',
 'N4698218021',
 'T3156402458',
 'I2915540327',
 'O4109219298',
 'T3677023494',
 'V3625380650',
 'J3936916362',
 'W4376557114',
 'H2029561900',
 'J4668796617',
 'T4697530761',
 'Y3943849716',
 'X3068907230',
 'I4466408838',
 'N4717040803',
 'N3753754373',
 'L4461086364',
 'P2128839717',
 'L4505436857',
 'M1001461571']

### product & brand & model & maker

상품 이름과 브랜드와 model의 관계를 분석해 볼 필요가 있다. 


가설 1) 
> 브랜드와 maker 사이의 관계 

가설 2) 
> 상품 이름과 model 사이의 관계

In [5]:
products = list_up_string_in_file('product')
brands = list_up_string_in_file('brand')
models = list_up_string_in_file('model')
makers = list_up_string_in_file('maker')
for p_name, b_name, m_name, ma_name in zip(products, brands, models, makers):
    print("product : ", p_name)
    print("brand : ", b_name)
    print("model : ", m_name)
    print("maker : ", ma_name)
    print("---------------------")

product :  직소퍼즐 - 1000조각 바다거북의 여행 (PL1275)
brand :  퍼즐라이프
model :  퍼즐라이프 직소퍼즐 바다거북의 여행
maker :  상품상세설명 참조
---------------------
product :  [모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][무료배송]
brand :  바보사랑
model :  아이폰6S/6S+ tree farm101 - 다이어리케이스|아이폰6S/6S+
maker :  MORY|해당없음
---------------------
product :  크리비아 기모 3부 속바지 GLG4314P
brand :  크리비아
model :  크리비아 기모 3부 속바지 GLG4314P
maker :  
---------------------
product :  [하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA
brand :  잭앤질
model :  [잭앤질] 남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA
maker :  ㈜크리스패션
---------------------
product :  코드프리혈당시험지50매/코드프리시험지/최장유효기간
brand :  
model :  SD코드프리혈당시험지[50매]
maker :  기타
---------------------
product :  아트박스 POOM/낭만창고  idk385-시원한 맥주 캬하~
brand :  
model :  아트박스 POOM/낭만창고  idk385-시원한 맥주 캬하~
maker :  
---------------------
product :  데버스 뉴 캠핑 BBQ 글러브 DVC E1209N 캠핑 등산
brand :  
model :  기타
maker :  기타
---------------------
product :  엘르스포츠 여성 비키니2PCS ETFLB06NVY
brand :  엘르스포츠
model :  ETFLB06NVY
maker :  기타
---

### Price

가격은 정수로 존재한다


가설 1) 
> 존재하지 않은 경우, -1로 되어 있는 것으로 추정된다

In [6]:
list(f['train']['price'][:20])

[16520,
 20370,
 -1,
 16280,
 -1,
 -1,
 -1,
 -1,
 30750,
 25200,
 24890,
 -1,
 45300,
 187800,
 -1,
 -1,
 -1,
 -1,
 14400,
 -1]

### updttm	

입력 시간은 중요하다고 느껴지지 않지만, 확인해볼 가치는 있다.

가설
> 20180227091029 : 2017년 02월 27일 09시 10분 29초 의 구조로 되어 있을 것이다

In [7]:
updttms = list_up_string_in_file('updttm')
for updttm in updttms :
    print(updttm)

20180227091029
20180429085019
20180426102314
20180422084512
20180424071623
20180426150317
20180422222810
20180420064936
20180419090423
20180401085848
20180419201828
20180410054511
20180421084431
20171123022655
20180328024813
20180322091807
20180427064917
20180402170347
20180420085522
20180426085358


### Category ID

카테고리 아이디도 정수 값으로 존재한다

가설 1) 
> 존재하지 않는 경우 -1로 되어 있는 것으로 추정된다.


In [8]:
bcateids = list(f['train']['bcateid'][:20])
mcateids = list(f['train']['mcateid'][:20])
scateids = list(f['train']['scateid'][:20])
dcateids = list(f['train']['dcateid'][:20])
print("[  대, 중,  소, 세]")
for b_id, m_id, s_id, d_id in zip(bcateids, mcateids, scateids, dcateids):
    print("[{:3d},{:3d},{:3d},{:3d}]".format(b_id, m_id,s_id, d_id))
    

[  대, 중,  소, 세]
[  1,  1,  2, -1]
[  3,  3,  4, -1]
[  5,  5,  6, -1]
[  7,  7,  8, -1]
[ 10,  9, 11, -1]
[ 11, 10, 12, -1]
[ 12, 11, 13, -1]
[ 13, 12, 14, -1]
[ 14, 13, 15, -1]
[ 11, 15, 17, -1]
[ 11, 16, 18, -1]
[ 14, 17, 19, -1]
[ 15, 18, 20,  3]
[ 16, 19, 21, -1]
[ 17, 20, 22, -1]
[ 18, 21, 23, -1]
[ 19, 22, 24, -1]
[  2, 23, 25, -1]
[ 21, 25, 27, -1]
[ 22, 26, 28, -1]


카테고리 정보는 따로 json 파일로 제공되었다.

In [9]:
json_data = open(os.path.join(RAW_DIR,'cate1.json')).read()
data = json.loads(json_data)

데이터는 key: 파일이름 - value : id로 구성되어 있는데, 

역으로 우리가 바꾸기 위해서는

key와 value를 서로 반대로 바꾸어주는 게 편하다

In [10]:
# b: 대분류 m: 중분류 s: 소분류 d: 세분류
b_cate_map = {value : key for key, value in data['b'].items()}
m_cate_map = {value : key for key, value in data['m'].items()}
s_cate_map = {value : key for key, value in data['s'].items()}
d_cate_map = {value : key for key, value in data['d'].items()}


In [11]:
print("[  대, 중,  소, 세]")
for b_id, m_id, s_id, d_id in zip(bcateids, mcateids, scateids, dcateids):
    print("[{},{},{},{}]".format(b_cate_map[b_id],
                                 m_cate_map[m_id],
                                 s_cate_map[s_id], 
                                 d_cate_map[d_id]))
    

[  대, 중,  소, 세]
[악기/취미/만들기,보드게임/퍼즐,직소/퍼즐,]
[휴대폰/액세서리,휴대폰액세서리,아이폰액세서리,]
[언더웨어,보정언더웨어,속바지/속치마,]
[남성의류,바지,일자면바지,]
[건강관리/실버용품,건강측정용품,혈당지,]
[홈/인테리어/가드닝,벽지/시트지/스티커,포인트스티커,]
[등산/캠핑/낚시,취사용품,기타취사용품,]
[구기/헬스/수영/스키,수영복,스포츠 비키니,]
[가방/지갑/잡화,여성가방,여성 클러치백,]
[홈/인테리어/가드닝,액자,액자세트,]
[홈/인테리어/가드닝,인테리어 시계,벽시계,]
[가방/지갑/잡화,지갑,명함/카드지갑,]
[계절가전/에어컨/온열기기,난방기기,온풍기,전기온풍기]
[주방/식기/용기,냄비/솥,냄비세트,]
[신발/수제화,여성플랫슈즈/로퍼,플랫,]
[스킨케어/메이크업,기능성화장품,영양보습,]
[홍삼/건강/다이어트식품,다이어트식품,다이어트보조식,]
[완구/교육/교구,퍼즐/감각발달완구,쌓기놀이,]
[수납/정리/선반,가구부품,가구손잡이,]
[유아동의류/신발/가방,패션소품,모자,]
